# Dataset Transform

This file will ask for a path to a directory which must contain:
- A file called _annotations.csv containing labels in a specific format. This will be outlined later in code when it will be altered to fit the Google Cloud Api input format.
- An amount of square jpg images of one set size. In my case 416x416 pixels.

---

### _annotations.csv input syntax

file_name, x1, y1, x2, y2, label

Where the x and y values are stored as integers.

### _annotations.csv output syntax

gs_file_path, label, x1, y1,,, x2, y2,,

Where the x and y values are now floating point numbers between 0 and 1. And ,, represents an empty column.

The gs_file_path represents where the images are stored in google cloud. But this can be set to whatever the filepaths
in the resulting csv file should be.

---

The purpose of this program will be to reduce the amount of images, and format the csv file.
The original dataset will be read only. And a new dataset will be created by creating a new csv file, and copying chosen images to a new destination.

To use this program, tweak the parameters below, and run all cells.

---

In [1]:
# import necessary libraries
import pandas as pd
import shutil

---

## Parameters

- src_path: Path to all source images and their _annotations.csv file.
- dest_path: Path to where the partition of images and the new _annotations.csv file should be sent.
- gs_filepath: Path to files in google cloud storage if using google cloud ML services.

---

- kept: The number of images to keep. (Note the resulting images may not be exactly the amount requested. There's a slight chance of collisions. For instance I requested 360 and resulted in 355.)
- move_images: Set to False if you only want the resulting csv file, and don't want the images to be moved to the destination folder.
- img_size: Size of square input images.

In [2]:
src_path = 'train/'
dest_path = 'train_aug/'
gs_filepath = 'gs://capstone_benchmark/'

In [ ]:
kept = 360
move_images = False
img_size = 416.

In [3]:
# read in the annotations file and label columns
labels = ['file_name', 'x1', 'y1', 'x2', 'y2', 'label']
ann = pd.read_csv(src_path + '_annotations.csv', sep=',', header=None, names=labels)
ann.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43473 entries, 0 to 43472
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  43473 non-null  object
 1   x1         43473 non-null  int64 
 2   y1         43473 non-null  int64 
 3   x2         43473 non-null  int64 
 4   y2         43473 non-null  int64 
 5   label      43473 non-null  object
dtypes: int64(4), object(2)
memory usage: 2.0+ MB


In [4]:
# For now we will drop all rows that don't have the label 'car'
ann = ann[ann['label'] == 'car']
print(ann['label'].unique())
ann.info()

['car']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32510 entries, 0 to 43472
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  32510 non-null  object
 1   x1         32510 non-null  int64 
 2   y1         32510 non-null  int64 
 3   x2         32510 non-null  int64 
 4   y2         32510 non-null  int64 
 5   label      32510 non-null  object
dtypes: int64(4), object(2)
memory usage: 1.7+ MB


In [7]:
# pick a random array of 'kept' images, these are the labels of the images we will
# transfer to the destination directory
img_subset = list(ann['file_name'].sample(n=kept, random_state=0))
ann_subset = ann[ann['file_name'].isin(img_subset)]
ann_subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2084 entries, 90 to 43354
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  2084 non-null   object
 1   x1         2084 non-null   int64 
 2   y1         2084 non-null   int64 
 3   x2         2084 non-null   int64 
 4   y2         2084 non-null   int64 
 5   label      2084 non-null   object
dtypes: int64(4), object(2)
memory usage: 114.0+ KB


In [10]:
name_mapping = dict()
count = 1
for name in ann_subset['file_name'].unique():
    name_mapping[name] = 'img' + str(count) + '.jpg'
    count += 1

In [16]:
ann_subset.head(2)

,file_name,x1,y1,x2,y2,label,new_name,empty
90,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,0.838942,0.403846,0.997596,0.701923,car,gs://capstone_benchmark/img1.jpg,NaN
91,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,0.067308,0.423077,0.235577,0.625000,car,gs://capstone_benchmark/img1.jpg,NaN


In [11]:
# copy files with unique file_name from source folder to name mapping in destination folder
if move_images:
    for name in ann_subset['file_name'].unique():
        shutil.copy(src_path + name, dest_path + name_mapping[name])

In [12]:
ann_subset['new_name'] = ann_subset['file_name'].map(name_mapping)
ann_subset['new_name'] = gs_filepath + ann_subset['new_name']
ann_subset.head(3)

C:\Users\19255\AppData\Local\Temp/ipykernel_2884/1532133406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ann_subset['new_name'] = ann_subset['file_name'].map(name_mapping)
C:\Users\19255\AppData\Local\Temp/ipykernel_2884/1532133406.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ann_subset['new_name'] = gs_filepath + ann_subset['new_name']


,file_name,x1,y1,x2,y2,label,new_name
90,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,349,168,415,292,car,gs://capstone_benchmark/img1.jpg
91,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,28,176,98,260,car,gs://capstone_benchmark/img1.jpg
92,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,91,180,118,240,car,gs://capstone_benchmark/img1.jpg


In [13]:
# append an empty column for padding requested data
ann_subset['empty'] = pd.Series()
ann_subset['x1'] = ann_subset['x1'].apply(lambda x: x / img_size)
ann_subset['y1'] = ann_subset['y1'].apply(lambda x: x / img_size)
ann_subset['x2'] = ann_subset['x2'].apply(lambda x: x / img_size)
ann_subset['y2'] = ann_subset['y2'].apply(lambda x: x / img_size)
ann_subset.head()

C:\Users\19255\AppData\Local\Temp/ipykernel_2884/2410846686.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ann_subset['empty'] = pd.Series()
C:\Users\19255\AppData\Local\Temp/ipykernel_2884/2410846686.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ann_subset['empty'] = pd.Series()
C:\Users\19255\AppData\Local\Temp/ipykernel_2884/2410846686.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,file_name,x1,y1,x2,y2,label,new_name,empty
90,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,0.838942,0.403846,0.997596,0.701923,car,gs://capstone_benchmark/img1.jpg,NaN
91,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,0.067308,0.423077,0.235577,0.625000,car,gs://capstone_benchmark/img1.jpg,NaN
92,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,0.218750,0.432692,0.283654,0.576923,car,gs://capstone_benchmark/img1.jpg,NaN
93,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,0.228365,0.439904,0.290865,0.588942,car,gs://capstone_benchmark/img1.jpg,NaN
94,1478732982753103095_jpg.rf.3c4c355e77fb688df76...,0.326923,0.461538,0.382212,0.526442,car,gs://capstone_benchmark/img1.jpg,NaN


In [14]:
# create final dataframe with columns in requested order
final_df = ann_subset[['new_name', 'label', 'x1', 'y1', 'empty', 'empty', 'x2', 'y2', 'empty', 'empty']]

# create an _annotations.csv file in the destination folder
final_df.to_csv(dest_path + '_annotations.csv', header=False, index=False)